In [1]:
pip install datasets pandas torch transformers[torch] python-dotenv peft

In [4]:
import os
from google.colab import userdata

# Get the Hugging Face token from Colab's secrets manager
HF_TOKEN = userdata.get('HF_TOKEN')

if HF_TOKEN:
    print("HF_TOKEN loaded successfully.")
else:
    print("HF_TOKEN not found. Please set it in Google Colab's secrets manager.")

HF_TOKEN loaded successfully.


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
login(token=HF_TOKEN)
model_name = "distilgpt2"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token
tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(model_name)
model

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
text = "ஒரு நாள் "
inputs = tokenizer(text, return_tensors="pt")
# Generate story
output = model.generate(inputs.input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ஒரு நாள் நாள் நாள் நாள் நாள் நாள் நாள் நாள் நாள


In [7]:
from datasets import load_dataset
# Load English-to-Colloquial Tamil dataset
raw_data = load_dataset("tniranjan/aitamilnadu_tamil_stories_no_instruct", split="train[:1000]")
data = raw_data.train_test_split(train_size=0.95)
data

README.md:   0%|          | 0.00/628 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/4.34M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/434k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1081 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/121 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 950
    })
    test: Dataset({
        features: ['text'],
        num_rows: 50
    })
})

In [8]:
tokenizer.pad_token = tokenizer.eos_token
def preprocess_batch(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=200)
tokenized_dataset = data.map(
preprocess_batch,
batched=True,
batch_size =4,
remove_columns=data["train"].column_names,)
# Print dataset details
print(tokenized_dataset)

Map:   0%|          | 0/950 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 950
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 50
    })
})


In [9]:
#Data collator
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
data_collator


DataCollatorForLanguageModeling(tokenizer=GPT2Tokenizer(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
), mlm=False, whole_word_mask=False, mlm_probability=0.15, mask_replace_prob=0.8, random_replace_prob=0.1, pad_to_multiple_of=None, return_tensors='pt', seed=None)

In [10]:
from peft import get_peft_model, LoraConfig, TaskType
# Define LoRA Configuration
lora_config = LoraConfig(
    r=8,  # Rank of LoRA matrices (adjust for speed/memory tradeoff)
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout for stability
    bias="none",  # No extra bias parameters
    task_type=TaskType.CAUSAL_LM  # Since we're fine-tuning GPT-style models
)
# Apply LoRA to the model
model = get_peft_model(model, lora_config)
# Print trainable parameters
model.print_trainable_parameters()

trainable params: 147,456 || all params: 82,060,032 || trainable%: 0.1797


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [14]:
model.train() #This ensures layers like Dropout & BatchNorm are active during training.
from torch.optim import AdamW  # Correct import
# Define new optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)
from transformers import TrainingArguments, Trainer
import transformers # Import transformers to check version

print(f"Transformers version: {transformers.__version__}") # Diagnostic: Print transformers version

training_args = TrainingArguments(
output_dir="./output",
# evaluation_strategy="epoch", # Temporarily commented out to resolve TypeError
#save_strategy="epoch",
save_steps=500,
learning_rate=1e-5,
weight_decay=0.01,
num_train_epochs=3,
per_device_train_batch_size=2,    # Batch size (adjust for GPU memory)
per_device_eval_batch_size=2,
logging_steps=50,
logging_dir="./logs",
resume_from_checkpoint=True
)
trainer = Trainer(
    model = model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    optimizers=(optimizer, None),
    data_collator=data_collator
)
trainer.train()

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Transformers version: 5.0.0


Step,Training Loss
50,1.251314
100,1.246538
150,1.254974
200,1.243108
250,1.221818
300,1.225775
350,1.221563
400,1.212085
450,1.224502
500,1.212402


TrainOutput(global_step=1425, training_loss=1.2092926627711247, metrics={'train_runtime': 82.3325, 'train_samples_per_second': 34.616, 'train_steps_per_second': 17.308, 'total_flos': 145952686080000.0, 'train_loss': 1.2092926627711247, 'epoch': 3.0})

In [17]:
text = "ஒரு நாள்"
inputs = tokenizer(text, return_tensors="pt")

# Move input tensors to the same device as the model
inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}

# Generate story
# Access input_ids and attention_mask using dictionary keys
output = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ஒரு நாள்்ர்ர்ர்ர்ர்ர்ர்ர்ர்ர்ர்ர்ர்ர்ர்ர்�


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM
trainer.save_model("/content/drive/My Drive/fine_tuned_distilgpt2_Tamil")
tokenizer.save_pretrained("/content/drive/My Drive/fine_tuned_distilgpt2_Tamil")

model = AutoModelForCausalLM.from_pretrained ("/content/drive/My Drive/fine_tuned_distilgpt2_200")
model
